In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import signal as scps
import os
import csv
import pytesseract


In [5]:
manga1 = cv2.imread("manga1.jpg")

In [2]:
def findSpeechBubbles(image):
    # Convert image to gray scale
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Recognizes rectangular/circular bubbles, struggles with dark colored bubbles 
    binary = cv2.threshold(imageGray,235,255,cv2.THRESH_BINARY)[1]
    # Find contours and document their heirarchy for later
    contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contourMap = {}
    finalContourList = []

    contourMap = filterContoursBySize(contours)
    contourMap = filterContainingContours(contourMap, hierarchy)

    # Sort final contour list
    finalContourList = list(contourMap.values())
    finalContourList.sort(key=lambda x:get_contour_precedence(x, binary.shape[1]))

    return finalContourList

def filterContoursBySize(contours):
    # We could pass this in and update it by reference, but I prefer this sort of 'immutable' handling.
    contourMap = {}

    for i in range(len(contours)):
        # Filter out speech bubble candidates with unreasonable size
        if cv2.contourArea(contours[i]) < 120000 and cv2.contourArea(contours[i]) > 4000:
            # Smooth out contours that were found
            epsilon = 0.0025*cv2.arcLength(contours[i], True)
            approximatedContour = cv2.approxPolyDP(contours[i], epsilon, True)
            contourMap[i] = approximatedContour

    return contourMap

# Sometimes the contour algorithm identifies entire panels, which can contain speech bubbles already
#  identified causing us to parse them twice via OCR. This method attempts to remove contours that 
#  contain other speech bubble candidate contours completely inside of them.
def filterContainingContours(contourMap, hierarchy):
    # I really wish there was a better way to do this than this O(n^2) removal of all parents in
    #  the heirarchy of a contour, but with the number of contours found this is the only way I can
    #  think of to do this.
    for i in list(contourMap.keys()):
        currentIndex = i
        while hierarchy[0][currentIndex][3] > 0:
            if hierarchy[0][currentIndex][3] in contourMap.keys():
                contourMap.pop(hierarchy[0][currentIndex][3])
            currentIndex = hierarchy[0][currentIndex][3]

    # I'd prefer to handle this 'immutably' like above, but I'd rather not make an unnecessary copy of the dict.
    return contourMap

def get_contour_precedence(contour, cols):
    tolerance_factor = 200
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

In [11]:
bubbles = findSpeechBubbles(manga1)
cv2.drawContours(manga1, bubbles, -1, (0, 255, 0), 3)
cv2.imshow('ImageWindow', manga1)
cv2.waitKey()

-1